In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from sklearn.metrics import accuracy_score, f1_score
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
import nltk
# nltk.download('words')

In [2]:
df = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_data.csv')
stp = np.load('stopwords.npy')
df.head()

,title,description,tags
0,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...,استانها
1,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...,استانها
2,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...,استانها
3,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...,یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...,اجتماعی
4,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...,اقتصادی


In [3]:
tags = list(df['tags'].unique())
tags = tags[:-1]

social = []
economics = []
iran = []
international = []
political = []
sc_cu_sp = []

for i in tags:
    if 'استان' in i :
        iran.append(i)
        tags.remove(i)

for i in tags:
    if 'استان ها ' in i :
        iran.append(i)
        tags.remove(i)
for i in tags:
    if 'ایران' in i :
        iran.append(i)
        tags.remove(i)
for i in tags:
    if 'ورزش' in i:
        sc_cu_sp.append(i)
        tags.remove(i)
for i in tags:
    if 'علم' in i:
        sc_cu_sp.append(i)
        tags.remove(i)
for i in tags:
    if 'اقتصاد' in i:
        economics.append(i)
        tags.remove(i)
for i in tags:
    if 'اقتصادی' in i:
        economics.append(i)
        tags.remove(i)
for i in tags:
    if 'سیاس' in i:
        political.append(i)
        tags.remove(i)
for i in tags:
    if 'اجتماع' in i:
        social.append(i)
        tags.remove(i)
for i in tags:
    if 'اجتماعی ' in i:
        social.append(i)
        tags.remove(i)
for i in tags:
    if 'فرهنگ' in i:
        sc_cu_sp.append(i)
        tags.remove(i)
for i in tags:
    if 'بین' in i:
        international.append(i)
        tags.remove(i)
for i in tags:
    if 'فرهنگی و هنری ' in i:
        sc_cu_sp.append(i)
        tags.remove(i)

In [4]:
sc_cu_sp

['ورزشی',
 'ورزش',
 'ورزشی > فوتبال، فوتسال',
 'ورزشی > سایر ورزش\u200cها',
 'ورزشی > کشتی، رزمی',
 'ورزشی > توپ و تور',
 'ورزشی > جهان ورزش',
 'علمی و دانشگاهی > صنفی، فرهنگی',
 'علمی و دانشگاهی > پژوهش',
 'علمی و دانشگاهی > علم و فناوری جهان',
 'علمی و دانشگاهی > آموزش',
 'فرهنگی',
 'فرهنگ و هنر',
 'فرهنگی و هنری > گردشگری و میراث',
 'فرهنگی و هنری > سینما و تئاتر',
 'فرهنگی و هنری > تجسمی و موسیقی',
 'فرهنگی و هنری > ادبیات و کتاب',
 'فرهنگی و هنری > رسانه',
 'فرهنگی و هنری > دین و اندیشه']

In [5]:
iran_left = ['استان ها > ایلام', 'ايران']
eco_left = ['بازار']
inter_left = ['الشرق الأوسط', 'آلمانی پیش\u200cرفته', 'آلمان', 'بین الملل > آسیا،خاورمیانه']
sc_left = ['ورزشی > ورزش بانوان', 'دانش و فناوری', 'گرافیک > اینفوگرافیک', 'علمی و دانشگاهی > جهاد دانشگاهی', 'فرهنگی و هنری > فرهنگ حماسه']


for i in iran_left:
    iran.append(i)
for i in eco_left:
    economics.append(i)
for i in inter_left:
    international.append(i)
for i in sc_left:
    sc_cu_sp.append(i)

In [6]:
tgs = df['tags']

final_tags = []

for i in tgs:
    if i in iran:
        final_tags.append('ایران_استانها')
    elif i in economics:
        final_tags.append('اقتصادی')
    elif i in sc_cu_sp:
        final_tags.append('علمی_فرهنگی_ورزشی')
    elif i in social:
        final_tags.append('اجتماعی')
    elif i in political:
        final_tags.append('سیاسی')
    elif i in international:
        final_tags.append('بین الملل')
    else:
        final_tags.append('None')


In [7]:
df['tags'] = final_tags

In [8]:
df = df[df['tags'] != 'None']

In [9]:
df.head()

,title,description,tags
0,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...,ایران_استانها
1,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...,ایران_استانها
2,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...,ایران_استانها
3,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...,یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...,اجتماعی
4,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...,اقتصادی


In [10]:
def removeNonPersian(x):
    words = ['quot', '&']
    filteredSentence = " ".join(w for w in nltk.wordpunct_tokenize(x)                                 if w.lower() not in words)
    return filteredSentence

df['title'] = df['title'].apply(removeNonPersian)

In [11]:
df = df.replace('None', 'ایران_استانها')
y = df['tags']
df.head()

,title,description,tags
0,روایتی از تحصیل زیرسقف ‌ های لرزان و سرمای سوز...,گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...,ایران_استانها
1,انقلابی سرشناس بحرین در گفت ‌ وگو با تسنیم : م...,گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...,ایران_استانها
2,‌ زندگی مردم در شهر قم ‌ جریان دارد / پیشگیری ...,گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...,ایران_استانها
3,واکنش ; پروفسور کرمی ; به دروغ ‌ پراکنی برخی ر...,یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...,اجتماعی
4,مردم نگران تأمین کالاهای موردنیاز خود نباشند /...,رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...,اقتصادی


In [12]:
df['sum'] = df['title'] + df['description']
# stp is an array of stop words
df['sum'] = df['sum'].apply(lambda x : " ".join(w for w in x.split() if w not in stp))

In [13]:
df = df.replace('اجتماعی', 0)
df = df.replace('اقتصادی', 1)
df = df.replace('ایران_استانها', 2)
df = df.replace('بین الملل', 3)
df = df.replace('سیاسی', 4)
df = df.replace('علمی_فرهنگی_ورزشی', 5)

In [20]:
X = df['sum']
y = df['tags']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
cv = TfidfVectorizer()
cv.fit(X_train)
# X = X.toarray()

TfidfVectorizer()

In [23]:
X_train = cv.fit_transform(X_train)
X_train = X_train.toarray()

In [24]:
X_test = cv.transform(X_test)
X_test = X_test.toarray()

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [4, 6, 10, 12, 15],
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': [2, 3, 5],
    'min_samples_split': [2, 3, 5]
}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


In [28]:
model = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, min_samples_split=2, max_depth=None, criterion='gini')
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=2)

In [30]:
pred = model.predict(X_test)
print(f'f1 : {f1_score(y_test, pred, average=None)[2]}')

f1 : 0.8667601683029453


In [ ]:
# train on the whole data
X = df['sum']
y = df['tags']


cv = TfidfVectorizer()
cv.fit(X)
X = X.toarray()
model.fit(X, y)

In [35]:
df_test['title'] = df_test['title'].apply(removeNonPersian)

df_test['sum'] = df_test['title'] + df_test['description']
# stp is an array of stop words
df_test['sum'] = df_test['sum'].apply(lambda x : " ".join(w for w in x.split() if w not in stp))

In [ ]:
test = cv.transform(df_test['sum'])
test = test.toarray()

In [ ]:
pred = model.predict(df_test)
df = pd.DataFrame({'res': pred})
df.to_csv('res.csv')